import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import re

# =========================
# USER SETTINGS
# =========================
USER_COUNTRY = "Philippines"
USER_COLOR = "teal"

HIGHLIGHT_COUNTRY = "Japan"
HIGHLIGHT_COLOR = "orange"

# =========================
# FETCH NATIONAL DEBT DATA
# =========================
url = "https://worldpopulationreview.com/country-rankings/countries-by-national-debt"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")
table = soup.find("table")
rows = table.find_all("tr")[1:]

DEBT_MULT = {"T": 1e12, "B": 1e9, "M": 1e6}
records = []

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 5:
        continue

    country = cols[1].text.strip()
    if country.upper() == "TOTAL":
        continue

    debt_text = cols[2].text.strip()
    gdp_pct_text = cols[3].text.strip()
    per_capita_text = cols[4].text.strip()

    m = re.match(r"\$(\d+(\.\d+)?)([TBM])", debt_text)
    if not m:
        continue

    debt_val = float(m.group(1))
    debt_unit = m.group(3)
    debt_usd = int(debt_val * DEBT_MULT[debt_unit])

    debt_pct_gdp = float(gdp_pct_text.replace("%", "")) if "%" in gdp_pct_text else None

    pc = None
    pc_text = per_capita_text.replace("$", "").replace(",", "").strip()
    pc_match = re.match(r"(\d+(\.\d+)?)\s*(Mn|Bn)?", pc_text)
    if pc_match:
        v = float(pc_match.group(1))
        u = pc_match.group(3)
        if u == "Mn":
            pc = int(v * 1_000_000)
        elif u == "Bn":
            pc = int(v * 1_000_000_000)
        else:
            pc = int(v)

    records.append({
        "Country": country,
        "Debt Label": debt_text,
        "Debt USD": debt_usd,
        "Debt % GDP": debt_pct_gdp,
        "Debt Per Capita": pc
    })

df = pd.DataFrame(records)

# =========================
# HELPERS
# =========================
def format_value_units(val, metric):
    if metric == "Debt % GDP":
        return f"{val}%"
    if val >= 1e12:
        return f"${val/1e12:.2f} T"
    if val >= 1e9:
        return f"${val/1e9:.2f} B"
    if val >= 1e6:
        return f"${val/1e6:.2f} M"
    return f"${val:,}"

def create_rank_df(df, col):
    d = df.dropna(subset=[col]).sort_values(col, ascending=False).reset_index(drop=True)
    d["Rank"] = d.index + 1
    return d.sort_values(col)

def bar_colors(d):
    return [
        USER_COLOR if c == USER_COUNTRY
        else HIGHLIGHT_COLOR if c == HIGHLIGHT_COUNTRY
        else "#1f77b4"
        for c in d["Country"]
    ]

def text_colors(d):
    return [
        USER_COLOR if c == USER_COUNTRY
        else HIGHLIG
